In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
from datetime import datetime
import time
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import plotly.express as px
#%matplotlib inline
#plt.style.use('seaborn')
from sklearn.cluster import AgglomerativeClustering as AC

In [2]:
df=pd.read_csv(r'C:\Users\ayish\GuviProjects\Crime data Project\Crimes_-_2001_to_Present.csv')

In [3]:
df.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,5741943,HN549294,08/25/2007 09:22:18 AM,074XX N ROGERS AVE,0560,ASSAULT,SIMPLE,OTHER,False,False,...,49.0,1.0,08A,NaN,NaN,2007,08/17/2015 03:03:40 PM,NaN,NaN,NaN
1,25953,JE240540,05/24/2021 03:06:00 PM,020XX N LARAMIE AVE,0110,HOMICIDE,FIRST DEGREE MURDER,STREET,True,False,...,36.0,19.0,01A,1141387.0,1913179.0,2021,11/18/2023 03:39:49 PM,41.917838,-87.755969,"(41.917838056, -87.755968972)"
2,26038,JE279849,06/26/2021 09:24:00 AM,062XX N MC CORMICK RD,0110,HOMICIDE,FIRST DEGREE MURDER,PARKING LOT,True,False,...,50.0,13.0,01A,1152781.0,1941458.0,2021,11/18/2023 03:39:49 PM,41.995219,-87.713355,"(41.995219444, -87.713354912)"
3,13279676,JG507211,11/09/2023 07:30:00 AM,019XX W BYRON ST,0620,BURGLARY,UNLAWFUL ENTRY,APARTMENT,False,False,...,47.0,5.0,05,1162518.0,1925906.0,2023,11/18/2023 03:39:49 PM,41.952345,-87.677975,"(41.952345086, -87.677975059)"
4,13274752,JG501049,11/12/2023 07:59:00 AM,086XX S COTTAGE GROVE AVE,0454,BATTERY,"AGGRAVATED P.O. - HANDS, FISTS, FEET, NO / MIN...",SMALL RETAIL STORE,True,False,...,6.0,44.0,08B,1183071.0,1847869.0,2023,12/09/2023 03:41:24 PM,41.737751,-87.604856,"(41.737750767, -87.604855911)"


In [4]:
df.columns

Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate',
       'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude',
       'Location'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8077151 entries, 0 to 8077150
Data columns (total 22 columns):
 #   Column                Dtype  
---  ------                -----  
 0   ID                    int64  
 1   Case Number           object 
 2   Date                  object 
 3   Block                 object 
 4   IUCR                  object 
 5   Primary Type          object 
 6   Description           object 
 7   Location Description  object 
 8   Arrest                bool   
 9   Domestic              bool   
 10  Beat                  int64  
 11  District              float64
 12  Ward                  float64
 13  Community Area        float64
 14  FBI Code              object 
 15  X Coordinate          float64
 16  Y Coordinate          float64
 17  Year                  int64  
 18  Updated On            object 
 19  Latitude              float64
 20  Longitude             float64
 21  Location              object 
dtypes: bool(2), float64(7), int64(3), object(1

In [7]:
df.isna().sum()

ID                           0
Case Number                  0
Date                         0
Block                        0
IUCR                         0
Primary Type                 0
Description                  0
Location Description     13087
Arrest                       0
Domestic                     0
Beat                         0
District                    47
Ward                    614846
Community Area          613470
FBI Code                     0
X Coordinate             88958
Y Coordinate             88958
Year                         0
Updated On                   0
Latitude                 88958
Longitude                88958
Location                 88958
dtype: int64

In [8]:
df.shape

(8077151, 22)

Number of distinct crimes:


In [9]:
crimes= df['Primary Type'].unique()
print("No of crimes:",len(crimes))
print()
print("Distinct crimes:",crimes)

No of crimes: 36

Distinct crimes: ['ASSAULT' 'HOMICIDE' 'BURGLARY' 'BATTERY' 'THEFT' 'CRIMINAL DAMAGE'
 'DECEPTIVE PRACTICE' 'CRIMINAL SEXUAL ASSAULT'
 'OFFENSE INVOLVING CHILDREN' 'MOTOR VEHICLE THEFT' 'ROBBERY'
 'SEX OFFENSE' 'OTHER OFFENSE' 'WEAPONS VIOLATION' 'STALKING'
 'CRIMINAL TRESPASS' 'PROSTITUTION' 'ARSON' 'NARCOTICS' 'KIDNAPPING'
 'CONCEALED CARRY LICENSE VIOLATION' 'INTERFERENCE WITH PUBLIC OFFICER'
 'PUBLIC PEACE VIOLATION' 'OBSCENITY' 'LIQUOR LAW VIOLATION'
 'INTIMIDATION' 'HUMAN TRAFFICKING' 'GAMBLING' 'CRIM SEXUAL ASSAULT'
 'OTHER NARCOTIC VIOLATION' 'NON-CRIMINAL' 'PUBLIC INDECENCY' 'RITUALISM'
 'DOMESTIC VIOLENCE' 'NON-CRIMINAL (SUBJECT SPECIFIED)' 'NON - CRIMINAL']


Dealing with missing values:

In [11]:
print("Number of missing values:", df.isna().sum().sum())

Number of missing values: 1686240


In [38]:
missing_values=list(df.isna().sum())
cols=list(df.columns)
col_final=[]
for i in range(len(cols)):
    if missing_values[i]==0:
        cols[i]="Others"
#print(cols)
d=dict(zip(cols, missing_values))
missing_vals=pd.DataFrame(d, index=['Missing values'])
dff=missing_vals.T
dff

,Missing values
Others,0
Location Description,13087
District,47
Ward,614846
Community Area,613470
X Coordinate,88958
Y Coordinate,88958
Latitude,88958
Longitude,88958
Location,88958


In [41]:


fig=px.bar(dff,title="Missing vales in the dataset",labels={"index":"columns","value":"count"})
fig.show()